# **소회**
두번째 모델로 돌린 것을 참조해보면, 팀의 베이스라인 모델로 검토가 가능할 듯 함

버트 모델에 대한 이해가 부족해서 클로드를 통해서 전체를 구성하고, gemini를 통해서 수정했음.

전체적인 이해도는 아직 부족한 상황임.

전제 구성을 효율적으로 하기 위하여, 모듈화된 팀의 코드는 사용하지 않고 재구성해봤음.

아웃풋은 predition.csv로 나오게 현재는 되어 있으니 수정하면 될 듯함.

In [ ]:
# 코랩 구글 드라이브 환경에서 하기 위하여 셋팅. 각 환경에 맞게 수정 요망
from getpass import getpass
token = getpass('GitHub token 입력: ')
!git clone https://{token}@github.com/hiyslee/DLthon_pepero_day.git

GitHub token 입력: ··········
Cloning into 'DLthon_pepero_day'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 94 (delta 33), reused 73 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (94/94), 2.79 MiB | 19.27 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
# 각 라이브러리 설치 중 에러 발생하여서 따로 빼냄
from transformers import AutoTokenizer
from torch.optim import AdamW # Corrected import from transformers.optimization to torch.optim

In [ ]:
# 옵티마이저 아담 설치 중 에러 발생으로 작업
# Install a specific version of transformers where AdamW is available in transformers.optimization
!pip install transformers==4.30.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 27.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 123.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [ ]:
#첫번째 버트 모델 - train 데이터에서 val 데이터 미분리로 다시 수행 (참조만)
"""
🎯 직장 내 대화 분류 시스템 - Google Colab 통합 코드
train.csv → 모델 학습 → test.csv 분류 → predictions.csv 저장 (Google Drive)
"""

# ============================================================================
# 01. Google Drive 마운트
# ============================================================================
print("="*70)
print("01. Google Drive 마운트")
print("="*70)

from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive 마운트 완료\n")

# ============================================================================
# 02. 파일 업로드 (train.csv, test.csv) - 이미 업로드됨
# ============================================================================
print("="*70)
print("02. 파일 업로드")
print("="*70)

from google.colab import files # Files already uploaded
print("train.csv와 test.csv를 선택하여 업로드하세요:")
uploaded = files.upload()
print(f"✓ 업로드된 파일: {list(uploaded.keys())}\n")
# print("✓ 파일이 이미 업로드되었습니다.\n") # Added message

# ============================================================================
# 03. 필요한 라이브러리 설치 - transformers version specific install handled in previous cell
# ============================================================================
print("="*70)
print("03. 라이브러리 설치")
print("="*70)

import subprocess
import sys

packages = ['torch', 'scikit-learn', 'tqdm', 'pandas'] # Removed transformers from this list
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("✓ 모든 라이브러리 설치 완료\n")

# ============================================================================
# 04. 라이브러리 임포트
# ============================================================================
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW # Corrected import location
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 05. 설정값
# ============================================================================
print("="*70)
print("04. 설정")
print("="*70)

# Corrected filenames based on uploaded files and assumed location
TRAIN_PATH = 'train.csv'
TEST_PATH = 'test.csv'
OUTPUT_PATH = '/content/drive/My Drive/predictions.csv'

MODEL_NAME = 'klue/bert-base'
BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"✓ 모델: {MODEL_NAME}")
print(f"✓ 디바이스: {DEVICE}")
print(f"✓ 배치 크기: {BATCH_SIZE}")
print(f"✓ 에포크: {EPOCHS}")
print(f"✓ 학습률: {LEARNING_RATE}\n")

# ============================================================================
# 06. Dataset 클래스 정의
# ============================================================================
print("="*70)
print("05. Dataset 클래스 정의")
print("="*70)

class ConversationDataset(Dataset):
    """대화 데이터를 처리하는 Dataset"""
    def __init__(self, conversations, tokenizer, labels=None, max_length=MAX_LENGTH):
        self.conversations = conversations
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        text = self.conversations[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)

        return item

print("✓ Dataset 클래스 정의 완료\n")

# ============================================================================
# 07. 학습 데이터 로드 및 전처리
# ============================================================================
print("="*70)
print("06. 학습 데이터 로드 및 전처리")
print("="*70)

df_train = pd.read_csv(TRAIN_PATH)
print(f"✓ 원본 데이터: {len(df_train):,}개")

# 결측치 제거
df_train = df_train.dropna(subset=['conversation', 'class'])
print(f"✓ 결측치 제거 후: {len(df_train):,}개")

# 중복 제거
df_train = df_train.drop_duplicates(subset=['conversation', 'class'])
print(f"✓ 중복 제거 후: {len(df_train):,}개")

print(f"\n📊 클래스 분포:")
print(df_train['class'].value_counts())
print()

# ============================================================================
# 08. 라벨 인코딩
# ============================================================================
print("="*70)
print("07. 라벨 인코딩")
print("="*70)

label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['class'])

print(f"✓ 클래스 개수: {len(label_encoder.classes_)}")
print(f"\n클래스 매핑:")
for i, class_name in enumerate(label_encoder.classes_):
    count = (df_train['class'] == class_name).sum()
    print(f"  {i}: {class_name:20s} ({count:,}개)")
print()

# ============================================================================
# 09. 토크나이저 로드
# ============================================================================
print("="*70)
print("08. 토크나이저 로드")
print("="*70)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"✓ 토크나이저 로드 완료\n")

# ============================================================================
# 10. 모델 로드
# ============================================================================
print("="*70)
print("09. 모델 로드")
print("="*70)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_encoder.classes_)
).to(DEVICE)
print(f"✓ 모델 로드 완료\n")

# ============================================================================
# 11. Dataset 및 DataLoader 생성
# ============================================================================
print("="*70)
print("10. Dataset 및 DataLoader 생성")
print("="*70)

train_dataset = ConversationDataset(
    df_train['conversation'].values,
    tokenizer,
    labels=df_train['label'].values
)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"✓ 학습 데이터셋: {len(train_dataset):,}개")
print(f"✓ DataLoader 생성 완료\n")

# ============================================================================
# 12. 옵티마이저 설정
# ============================================================================
print("="*70)
print("11. 옵티마이저 설정")
print("="*70)

# Corrected optimizer initialization for transformers version 4.30.0
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
print(f"✓ AdamW 옵티마이저 설정 완료\n")

# ============================================================
# 13. 모델 학습
# ============================================================
print("="*70)
print("12. 모델 학습 시작")
print("="*70 + "\n")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for batch in pbar:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계 계산
        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        # 진행 상황 업데이트
        pbar.set_postfix({
            'loss': f'{total_loss/len(pbar):.4f}',
            'acc': f'{correct/total:.4f}'
        })

    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total
    print(f"✓ Epoch {epoch+1} - Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}\n")

print("✓ 학습 완료!\n")

# ============================================================
# 14. 테스트 데이터 로드
# ============================================================
print("="*70)
print("13. 테스트 데이터 로드")
print("="*70)

df_test = pd.read_csv(TEST_PATH)
print(f"✓ 테스트 데이터: {len(df_test):,}개\n")

# ============================================================
# 15. 테스트 Dataset 및 DataLoader 생성
# ============================================================
print("="*70)
print("14. 테스트 Dataset 생성")
print("="*70)

test_dataset = ConversationDataset(
    df_test['conversation'].values,
    tokenizer,
    labels=None
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"✓ 테스트 DataLoader 생성 완료\n")

# ============================================================
# 16. 테스트 데이터 분류
# ============================================================
print("="*70)
print("15. 테스트 데이터 분류")
print("="*70 + "\n")

model.eval()
predictions = []
confidences = []

print("분류 중...\n")
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)

        # 예측 클래스 및 신뢰도
        pred_ids = torch.argmax(logits, dim=1)
        pred_classes = [label_encoder.classes_[pid] for pid in pred_ids.cpu().numpy()]
        conf_scores = torch.max(probs, dim=1)[0].cpu().numpy()

        predictions.extend(pred_classes)
        confidences.extend(conf_scores)

print("\n✓ 분류 완료!\n")

# ============================================================
# 17. 결과 데이터프레임 생성
# ============================================================
print("="*70)
print("16. 결과 데이터프레임 생성")
print("="*70)

df_test['prediction'] = predictions
df_test['confidence'] = confidences

print(f"✓ 결과 데이터프레임 생성 완료")
print(f"✓ 컬럼: {df_test.columns.tolist()}\n")

# ============================================================
# 18. 결과를 Google Drive에 저장
# ============================================================
print("="*70)
print("17. 결과 저장 (Google Drive)")
print("="*70)

df_test.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"✓ 저장 경로: {OUTPUT_PATH}")
print(f"✓ 저장된 행 수: {len(df_test):,}개\n")

# ============================================================
# 19. 결과 요약 - 클래스 분포
# ============================================================
print("="*70)
print("18. 결과 요약 - 클래스 분포")
print("="*70)

print(f"\n✓ 클래스별 분포:")
print(df_test['prediction'].value_counts())
print()

# ============================================================
# 20. 결과 요약 - 신뢰도 통계
# ============================================================
print("="*70)
print("19. 결과 요약 - 신뢰도 통계")
print("="*70)

print(f"\n✓ 신뢰도 통계:")
print(f"  평균: {df_test['confidence'].mean():.4f}")
print(f"  최고: {df_test['confidence'].max():.4f}")
print(f"  최저: {df_test['confidence'].min():.4f}")
print(f"  표준편차: {df_test['confidence'].std():.4f}\n")

# ============================================================
# 21. 결과 요약 - 신뢰도 등급
# ============================================================
print("="*70)
print("20. 결과 요약 - 신뢰도 등급")
print("="*70)

high_conf = (df_test['confidence'] >= 0.9).sum()
mid_conf = ((df_test['confidence'] >= 0.7) & (df_test['confidence'] < 0.9)).sum()
low_conf = (df_test['confidence'] < 0.7).sum()

print(f"\n✓ 신뢰도 등급:")
print(f"  높음 (≥0.9):  {high_conf:3d}개 ({high_conf/len(df_test)*100:5.1f}%)")
print(f"  중간 (0.7~0.9): {mid_conf:3d}개 ({mid_conf/len(df_test)*100:5.1f}%)")
print(f"  낮음 (<0.7):  {low_conf:3d}개 ({low_conf/len(df_test)*100:5.1f}%)\n")

# ============================================================
# 22. 상위 10개 결과 출력
# ============================================================
print("="*70)
print("21. 상위 10개 예측 결과")
print("="*70)

print("\n")
print(df_test[['idx', 'prediction', 'confidence']].head(10).to_string(index=False))
print()

# ============================================================
# 23. 신뢰도 낮은 결과 (< 0.7)
# ============================================================
print("="*70)
print("22. 신뢰도 낮은 결과 (< 0.7)")
print("="*70)

low_confidence_df = df_test[df_test['confidence'] < 0.7].sort_values('confidence')

if len(low_confidence_df) > 0:
    print(f"\n✓ 신뢰도 낮은 상위 5개:")
    print(low_confidence_df[['idx', 'prediction', 'confidence']].head(5).to_string(index=False))
else:
    print("\n✓ 신뢰도가 0.7 이상인 예측만 있습니다!")

print()

# ============================================================
# 24. 전체 예측 결과 출력
# ============================================================
print("="*70)
print("23. 전체 예측 결과 (샘플)")
print("="*70)

print("\n✓ 처음 5개:")
print(df_test[['idx', 'conversation', 'prediction', 'confidence']].head(5).to_string(index=False))

print("\n\n✓ 마지막 5개:")
print(df_test[['idx', 'conversation', 'prediction', 'confidence']].tail(5).to_string(index=False))

# ============================================================
# 25. 최종 완료 메시지
# ============================================================
print("\n\n" + "█"*70)
print("█" + " "*68 + "█")
print("█" + "  ✅ 모든 작업이 완료되었습니다!".center(68) + "█")
print("█" + " "*68 + "█")
print("█"*70)

print("\n📁 결과 파일 정보:")
print(f"  📊 파일 이름: predictions.csv")
print(f"  📍 저장 위치: My Drive/predictions.csv")
print(f"  📋 총 행 수: {len(df_test):,}개")
print(f"  📌 컬럼: {', '.join(df_test.columns.tolist())}")

print("\n🎯 결과 컬럼 설명:")
print(f"  • idx: 원본 테스트 데이터의 인덱스")
print(f"  • conversation: 원본 대화 텍스트")
print(f"  • prediction: 모델이 예측한 클래스")
print(f"  • confidence: 예측의 신뢰도 (0~1, 1에 가까울수록 신뢰성 높음)")

print("\n💾 Google Drive에서 다운로드하거나 확인할 수 있습니다!")
print("\n" + "█"*70 + "\n")

01. Google Drive 마운트
Mounted at /content/drive
✓ Google Drive 마운트 완료

02. 파일 업로드
train.csv와 test.csv를 선택하여 업로드하세요:


Saving test.csv to test.csv
Saving train.csv to train.csv
✓ 업로드된 파일: ['test.csv', 'train.csv']

03. 라이브러리 설치
✓ 모든 라이브러리 설치 완료

04. 설정
✓ 모델: klue/bert-base
✓ 디바이스: cuda
✓ 배치 크기: 32
✓ 에포크: 3
✓ 학습률: 2e-05

05. Dataset 클래스 정의
✓ Dataset 클래스 정의 완료

06. 학습 데이터 로드 및 전처리
✓ 원본 데이터: 4,950개
✓ 결측치 제거 후: 4,950개
✓ 중복 제거 후: 4,619개

📊 클래스 분포:
class
기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
일반 대화           773
Name: count, dtype: int64

07. 라벨 인코딩
✓ 클래스 개수: 5

클래스 매핑:
  0: 갈취 대화                (973개)
  1: 기타 괴롭힘 대화            (1,011개)
  2: 일반 대화                (773개)
  3: 직장 내 괴롭힘 대화          (970개)
  4: 협박 대화                (892개)

08. 토크나이저 로드


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

✓ Epoch 2 - Loss: 0.1911 | Accuracy: 0.9396



Epoch 3/3: 100%|██████████| 145/145 [06:48<00:00,  2.82s/it, loss=0.0930, acc=0.9738]


✓ Epoch 3 - Loss: 0.0930 | Accuracy: 0.9738

✓ 학습 완료!

13. 테스트 데이터 로드
✓ 테스트 데이터: 500개

14. 테스트 Dataset 생성
✓ 테스트 DataLoader 생성 완료

15. 테스트 데이터 분류

분류 중...



100%|██████████| 16/16 [00:14<00:00,  1.09it/s]



✓ 분류 완료!

16. 결과 데이터프레임 생성
✓ 결과 데이터프레임 생성 완료
✓ 컬럼: ['idx', 'conversation', 'prediction', 'confidence']

17. 결과 저장 (Google Drive)
✓ 저장 경로: /content/drive/My Drive/predictions.csv
✓ 저장된 행 수: 500개

18. 결과 요약 - 클래스 분포

✓ 클래스별 분포:
prediction
기타 괴롭힘 대화      147
갈취 대화          123
직장 내 괴롭힘 대화    120
협박 대화          103
일반 대화            7
Name: count, dtype: int64

19. 결과 요약 - 신뢰도 통계

✓ 신뢰도 통계:
  평균: 0.9303
  최고: 0.9974
  최저: 0.3771
  표준편차: 0.1265

20. 결과 요약 - 신뢰도 등급

✓ 신뢰도 등급:
  높음 (≥0.9):  404개 ( 80.8%)
  중간 (0.7~0.9):  56개 ( 11.2%)
  낮음 (<0.7):   40개 (  8.0%)

21. 상위 10개 예측 결과


  idx  prediction  confidence
t_000       갈취 대화    0.985439
t_001 직장 내 괴롭힘 대화    0.996254
t_002 직장 내 괴롭힘 대화    0.974581
t_003   기타 괴롭힘 대화    0.583188
t_004   기타 괴롭힘 대화    0.847511
t_005       협박 대화    0.979174
t_006       협박 대화    0.990989
t_007       갈취 대화    0.934574
t_008   기타 괴롭힘 대화    0.730735
t_009       갈취 대화    0.993708

22. 신뢰도 낮은 결과 (< 0.7)

✓ 신뢰도 낮은 상위 5개:
  idx prediction  confidence
t_176      협박 대화    

In [ ]:
# 2번째로 다시 수정 후 작업 - validation 분리
"""
🎯 직장 내 대화 분류 시스템 - Google Colab 통합 코드 (Validation 평가 포함)

데이터 분할:
  • train.csv (90%) → 모델 학습
  • train.csv (10%) → Validation 평가
  • test.csv → 테스트 분류

출력:
  • predictions.csv (테스트 분류 결과)
  • validation_report.txt (검증 평가 보고서)
"""


# ============================================================================
# 03. 필요한 라이브러리 설치
# ============================================================================
print("="*70)
print("03. 라이브러리 설치")
print("="*70)

import subprocess
import sys

packages = ['torch', 'transformers', 'scikit-learn', 'tqdm', 'pandas', 'numpy']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("✓ 모든 라이브러리 설치 완료\n")

# ============================================================================
# 04. 라이브러리 임포트
# ============================================================================
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 05. 설정값
# ============================================================================
print("="*70)
print("04. 설정")
print("="*70)

TRAIN_PATH = 'train.csv'
TEST_PATH = 'test.csv'
OUTPUT_PATH = '/content/drive/My Drive/predictions.csv'
VALIDATION_REPORT_PATH = '/content/drive/My Drive/validation_report.txt'

MODEL_NAME = 'klue/bert-base'
BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
VALIDATION_RATIO = 0.1
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"✓ 모델: {MODEL_NAME}")
print(f"✓ 디바이스: {DEVICE}")
print(f"✓ 배치 크기: {BATCH_SIZE}")
print(f"✓ 에포크: {EPOCHS}")
print(f"✓ 학습률: {LEARNING_RATE}")
print(f"✓ Validation 비율: {VALIDATION_RATIO*100}%\n")

# ============================================================================
# 06. Dataset 클래스 정의
# ============================================================================
print("="*70)
print("05. Dataset 클래스 정의")
print("="*70)

class ConversationDataset(Dataset):
    def __init__(self, conversations, tokenizer, labels=None, max_length=MAX_LENGTH):
        self.conversations = conversations
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        text = self.conversations[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)

        return item

print("✓ Dataset 클래스 정의 완료\n")

# ============================================================================
# 07. 학습 데이터 로드 및 전처리
# ============================================================================
print("="*70)
print("06. 학습 데이터 로드 및 전처리")
print("="*70)

df_train = pd.read_csv(TRAIN_PATH)
print(f"✓ 원본 데이터: {len(df_train):,}개")

df_train = df_train.dropna(subset=['conversation', 'class'])
print(f"✓ 결측치 제거 후: {len(df_train):,}개")

df_train = df_train.drop_duplicates(subset=['conversation', 'class'])
print(f"✓ 중복 제거 후: {len(df_train):,}개")

print(f"\n📊 클래스 분포:")
print(df_train['class'].value_counts())
print()

# ============================================================================
# 08. 라벨 인코딩
# ============================================================================
print("="*70)
print("07. 라벨 인코딩")
print("="*70)

label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['class'])

print(f"✓ 클래스 개수: {len(label_encoder.classes_)}")
print(f"\n클래스 매핑:")
for i, class_name in enumerate(label_encoder.classes_):
    count = (df_train['class'] == class_name).sum()
    print(f"  {i}: {class_name:20s} ({count:,}개)")
print()

# ============================================================================
# 09. 학습/검증 데이터 분할 (90/10)
# ============================================================================
print("="*70)
print("08. 학습/검증 데이터 분할 (90/10)")
print("="*70)

X_train, X_val, y_train, y_val, class_train, class_val = train_test_split(
    df_train['conversation'].values,
    df_train['label'].values,
    df_train['class'].values,
    test_size=VALIDATION_RATIO,
    random_state=42,
    stratify=df_train['label'].values
)

print(f"✓ 학습 데이터: {len(X_train):,}개 (90%)")
print(f"✓ 검증 데이터: {len(X_val):,}개 (10%)\n")

# ============================================================================
# 10. 토크나이저 로드
# ============================================================================
print("="*70)
print("09. 토크나이저 로드")
print("="*70)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"✓ 토크나이저 로드 완료\n")

# ============================================================================
# 11. 모델 로드
# ============================================================================
print("="*70)
print("10. 모델 로드")
print("="*70)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_encoder.classes_)
).to(DEVICE)
print(f"✓ 모델 로드 완료\n")

# ============================================================================
# 12. 학습/검증 Dataset 및 DataLoader 생성
# ============================================================================
print("="*70)
print("11. 학습/검증 Dataset 생성")
print("="*70)

train_dataset = ConversationDataset(X_train, tokenizer, labels=y_train)
val_dataset = ConversationDataset(X_val, tokenizer, labels=y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print(f"✓ 학습 데이터셋: {len(train_dataset):,}개")
print(f"✓ 검증 데이터셋: {len(val_dataset):,}개")
print(f"✓ DataLoader 생성 완료\n")

# ============================================================================
# 13. 옵티마이저 설정
# ============================================================================
print("="*70)
print("12. 옵티마이저 설정")
print("="*70)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
print(f"✓ AdamW 옵티마이저 설정 완료\n")

# ============================================================================
# 14. 모델 학습 및 검증
# ============================================================================
print("="*70)
print("13. 모델 학습 시작 (학습/검증)")
print("="*70 + "\n")

best_val_accuracy = 0
best_model_state = None
training_history = []

for epoch in range(EPOCHS):
    # ========== 학습 단계 ==========
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Train")

    for batch in pbar:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix({
            'loss': f'{total_loss/len(pbar):.4f}',
            'acc': f'{correct/total:.4f}'
        })

    train_avg_loss = total_loss / len(train_loader)
    train_accuracy = correct / total

    # ========== 검증 단계 ==========
    model.eval()
    val_total_loss = 0
    val_correct = 0
    val_total = 0
    all_val_preds = []
    all_val_labels = []

    pbar_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Val  ")

    with torch.no_grad():
        for batch in pbar_val:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            val_total_loss += loss.item()

            predictions = torch.argmax(outputs.logits, dim=1)
            val_correct += (predictions == labels).sum().item()
            val_total += labels.size(0)

            all_val_preds.extend(predictions.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

            pbar_val.set_postfix({
                'loss': f'{val_total_loss/len(pbar_val):.4f}',
                'acc': f'{val_correct/val_total:.4f}'
            })

    val_avg_loss = val_total_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    # 히스토리 저장
    training_history.append({
        'epoch': epoch + 1,
        'train_loss': train_avg_loss,
        'train_acc': train_accuracy,
        'val_loss': val_avg_loss,
        'val_acc': val_accuracy
    })

    # 출력
    print(f"\n✓ Epoch {epoch+1}/{EPOCHS}")
    print(f"  Train - Loss: {train_avg_loss:.4f} | Accuracy: {train_accuracy:.4f}")
    print(f"  Val   - Loss: {val_avg_loss:.4f} | Accuracy: {val_accuracy:.4f}")

    # 최고 성능 모델 저장
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = model.state_dict().copy()
        print(f"  ⭐ 최고 검증 정확도 갱신! ({val_accuracy:.4f})")

    print()

# 최고 성능 모델 복원
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("✓ 최고 성능 모델로 복원 완료!\n")

# ============================================================================
# 15. 최종 검증 평가 (상세 지표)
# ============================================================================
print("="*70)
print("14. 최종 검증 평가 (상세 지표)")
print("="*70 + "\n")

model.eval()
final_val_preds = []
final_val_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        final_val_preds.extend(predictions.cpu().numpy())
        final_val_labels.extend(labels.cpu().numpy())

# 평가 지표 계산
val_accuracy = accuracy_score(final_val_labels, final_val_preds)
val_precision = precision_score(final_val_labels, final_val_preds, average='weighted', zero_division=0)
val_recall = recall_score(final_val_labels, final_val_preds, average='weighted', zero_division=0)
val_f1 = f1_score(final_val_labels, final_val_preds, average='weighted', zero_division=0)

print(f"✓ 검증 세트 성능 지표:")
print(f"  Accuracy:  {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print(f"  Precision: {val_precision:.4f} ({val_precision*100:.2f}%)")
print(f"  Recall:    {val_recall:.4f} ({val_recall*100:.2f}%)")
print(f"  F1-Score:  {val_f1:.4f} ({val_f1*100:.2f}%)\n")

# 클래스별 성능
print(f"✓ 클래스별 성능:")
val_pred_array = np.array(final_val_preds)
val_label_array = np.array(final_val_labels)

for i, class_name in enumerate(label_encoder.classes_):
    class_mask = val_label_array == i
    if class_mask.sum() > 0:
        class_accuracy = (val_pred_array[class_mask] == i).sum() / class_mask.sum()
        class_count = class_mask.sum()
        print(f"  {class_name:20s}: Accuracy={class_accuracy:.4f} (n={class_count})")

print()

# 혼동 행렬
print(f"✓ 혼동 행렬 (Confusion Matrix):")
conf_mat = confusion_matrix(final_val_labels, final_val_preds)
print(conf_mat)
print()

# ============================================================================
# 16. 테스트 데이터 로드
# ============================================================================
print("="*70)
print("15. 테스트 데이터 로드")
print("="*70)

df_test = pd.read_csv(TEST_PATH)
print(f"✓ 테스트 데이터: {len(df_test):,}개\n")

# ============================================================================
# 17. 테스트 Dataset 및 DataLoader 생성
# ============================================================================
print("="*70)
print("16. 테스트 Dataset 생성")
print("="*70)

test_dataset = ConversationDataset(
    df_test['conversation'].values,
    tokenizer,
    labels=None
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"✓ 테스트 DataLoader 생성 완료\n")

# ============================================================================
# 18. 테스트 데이터 분류
# ============================================================================
print("="*70)
print("17. 테스트 데이터 분류")
print("="*70 + "\n")

model.eval()
predictions = []
confidences = []

print("분류 중...\n")
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)

        pred_ids = torch.argmax(logits, dim=1)
        pred_classes = [label_encoder.classes_[pid] for pid in pred_ids.cpu().numpy()]
        conf_scores = torch.max(probs, dim=1)[0].cpu().numpy()

        predictions.extend(pred_classes)
        confidences.extend(conf_scores)

print("\n✓ 분류 완료!\n")

# ============================================================================
# 19. 결과 데이터프레임 생성
# ============================================================================
print("="*70)
print("18. 결과 데이터프레임 생성")
print("="*70)

df_test['prediction'] = predictions
df_test['confidence'] = confidences

print(f"✓ 결과 데이터프레임 생성 완료")
print(f"✓ 컬럼: {df_test.columns.tolist()}\n")

# ============================================================================
# 20. 결과를 Google Drive에 저장
# ============================================================================
print("="*70)
print("19. 결과 저장 (Google Drive)")
print("="*70)

df_test.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"✓ 예측 결과 저장: {OUTPUT_PATH}")
print(f"✓ 저장된 행 수: {len(df_test):,}개\n")

# ============================================================================
# 21. 검증 평가 보고서 생성 및 저장
# ============================================================================
print("="*70)
print("20. 검증 평가 보고서 생성 및 저장")
print("="*70)

report = f"""
{'='*80}
직장 내 대화 분류 모델 - 검증 평가 보고서
{'='*80}

📊 모델 설정
{'='*80}
모델: {MODEL_NAME}
배치 크기: {BATCH_SIZE}
에포크: {EPOCHS}
학습률: {LEARNING_RATE}
디바이스: {DEVICE}

📁 데이터 분할
{'='*80}
원본 학습 데이터: {len(df_train):,}개
↓ 분할 ↓
학습 데이터: {len(X_train):,}개 (90%)
검증 데이터: {len(X_val):,}개 (10%)
테스트 데이터: {len(df_test):,}개

🏷️  클래스 정보
{'='*80}
클래스 개수: {len(label_encoder.classes_)}

클래스 분포:
"""

for i, class_name in enumerate(label_encoder.classes_):
    train_count = (class_train == class_name).sum()
    val_count = (class_val == class_name).sum()
    total_count = train_count + val_count
    report += f"  {i}: {class_name:20s} - 학습: {train_count:,}개 | 검증: {val_count:,}개 | 총: {total_count:,}개\n"

report += f"""
📈 학습 이력
{'='*80}
"""

for hist in training_history:
    report += f"Epoch {hist['epoch']}: Train Loss={hist['train_loss']:.4f} | Train Acc={hist['train_acc']:.4f} | Val Loss={hist['val_loss']:.4f} | Val Acc={hist['val_acc']:.4f}\n"

report += f"""
{'='*80}
최고 검증 정확도: {best_val_accuracy:.4f}

📈 검증 세트 최종 성능 지표 (최고 성능 모델)
{'='*80}
Accuracy:  {val_accuracy:.4f} ({val_accuracy*100:.2f}%)
Precision: {val_precision:.4f} ({val_precision*100:.2f}%)
Recall:    {val_recall:.4f} ({val_recall*100:.2f}%)
F1-Score:  {val_f1:.4f} ({val_f1*100:.2f}%)

📊 클래스별 성능
{'='*80}
"""

for i, class_name in enumerate(label_encoder.classes_):
    class_mask = val_label_array == i
    if class_mask.sum() > 0:
        class_accuracy = (val_pred_array[class_mask] == i).sum() / class_mask.sum()
        class_count = class_mask.sum()
        report += f"{class_name:20s}: Accuracy={class_accuracy:.4f} (n={class_count})\n"

report += f"""
🔀 혼동 행렬 (Confusion Matrix)
{'='*80}
클래스: {list(label_encoder.classes_)}

"""

for i, class_name in enumerate(label_encoder.classes_):
    report += f"{class_name:20s}: {conf_mat[i].tolist()}\n"

report += f"""
🎯 테스트 세트 분류 결과
{'='*80}
총 테스트 데이터: {len(df_test):,}개

클래스별 분포:
"""

for pred_class in label_encoder.classes_:
    count = (df_test['prediction'] == pred_class).sum()
    percentage = (count / len(df_test)) * 100
    report += f"  {pred_class:20s}: {count:3d}개 ({percentage:5.1f}%)\n"

report += f"""
신뢰도 통계:
  평균: {df_test['confidence'].mean():.4f}
  최고: {df_test['confidence'].max():.4f}
  최저: {df_test['confidence'].min():.4f}
  표준편차: {df_test['confidence'].std():.4f}

신뢰도 등급:
"""

high_conf = (df_test['confidence'] >= 0.9).sum()
mid_conf = ((df_test['confidence'] >= 0.7) & (df_test['confidence'] < 0.9)).sum()
low_conf = (df_test['confidence'] < 0.7).sum()

report += f"  높음 (≥0.9):  {high_conf:3d}개 ({high_conf/len(df_test)*100:5.1f}%)\n"
report += f"  중간 (0.7~0.9): {mid_conf:3d}개 ({mid_conf/len(df_test)*100:5.1f}%)\n"
report += f"  낮음 (<0.7):  {low_conf:3d}개 ({low_conf/len(df_test)*100:5.1f}%)\n"

report += f"""
{'='*80}
생성 날짜: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*80}
"""

with open(VALIDATION_REPORT_PATH, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"✓ 검증 보고서 저장: {VALIDATION_REPORT_PATH}\n")

# ============================================================================
# 22. 결과 요약 - 클래스 분포
# ============================================================================
print("="*70)
print("21. 결과 요약 - 테스트 세트 클래스 분포")
print("="*70)

print(f"\n✓ 테스트 세트 클래스별 분포:")
print(df_test['prediction'].value_counts())
print()

# ============================================================================
# 23. 결과 요약 - 신뢰도 통계
# ============================================================================
print("="*70)
print("22. 결과 요약 - 신뢰도 통계")
print("="*70)

print(f"\n✓ 신뢰도 통계:")
print(f"  평균: {df_test['confidence'].mean():.4f}")
print(f"  최고: {df_test['confidence'].max():.4f}")
print(f"  최저: {df_test['confidence'].min():.4f}")
print(f"  표준편차: {df_test['confidence'].std():.4f}\n")

# ============================================================================
# 24. 결과 요약 - 신뢰도 등급
# ============================================================================
print("="*70)
print("23. 결과 요약 - 신뢰도 등급")
print("="*70)

print(f"\n✓ 신뢰도 등급:")
print(f"  높음 (≥0.9):  {high_conf:3d}개 ({high_conf/len(df_test)*100:5.1f}%)")
print(f"  중간 (0.7~0.9): {mid_conf:3d}개 ({mid_conf/len(df_test)*100:5.1f}%)")
print(f"  낮음 (<0.7):  {low_conf:3d}개 ({low_conf/len(df_test)*100:5.1f}%)\n")

# ============================================================================
# 25. 상위 10개 예측 결과
# ============================================================================
print("="*70)
print("24. 상위 10개 예측 결과")
print("="*70)

print("\n")
print(df_test[['idx', 'prediction', 'confidence']].head(10).to_string(index=False))
print()

# ============================================================================
# 26. 신뢰도 낮은 결과 (< 0.7)
# ============================================================================
print("="*70)
print("25. 신뢰도 낮은 결과 (< 0.7)")
print("="*70)

low_confidence_df = df_test[df_test['confidence'] < 0.7].sort_values('confidence')

if len(low_confidence_df) > 0:
    print(f"\n✓ 신뢰도 낮은 상위 5개:")
    print(low_confidence_df[['idx', 'prediction', 'confidence']].head(5).to_string(index=False))
else:
    print("\n✓ 신뢰도가 0.7 이상인 예측만 있습니다!")

print()

# ============================================================================
# 27. 전체 예측 결과 샘플
# ============================================================================
print("="*70)
print("26. 전체 예측 결과 (샘플)")
print("="*70)

print("\n✓ 처음 5개:")
print(df_test[['idx', 'conversation', 'prediction', 'confidence']].head(5).to_string(index=False))

print("\n\n✓ 마지막 5개:")
print(df_test[['idx', 'conversation', 'prediction', 'confidence']].tail(5).to_string(index=False))

# ============================================================================
# 28. 최종 완료 메시지
# ============================================================================
print("\n\n" + "█"*70)
print("█" + " "*68 + "█")
print("█" + "  ✅ 모든 작업이 완료되었습니다!".center(68) + "█")
print("█" + " "*68 + "█")
print("█"*70)

print("\n📁 저장된 파일 정보:")
print(f"\n  📊 파일 1: predictions.csv")
print(f"     위치: My Drive/predictions.csv")
print(f"     내용: 테스트 데이터 분류 결과")
print(f"     행 수: {len(df_test):,}개")
print(f"     컬럼: idx, conversation, prediction, confidence")

print(f"\n  📋 파일 2: validation_report.txt")
print(f"     위치: My Drive/validation_report.txt")
print(f"     내용: 검증 평가 상세 보고서")
print(f"     포함 정보:")
print(f"       - 모델 설정 및 데이터 분할")
print(f"       - 클래스별 분포")
print(f"       - 학습 이력 (에포크별)")
print(f"       - 검증 성능 지표")
print(f"       - 클래스별 성능 분석")
print(f"       - 혼동 행렬")
print(f"       - 테스트 분류 결과")

print(f"\n🎯 검증 세트 최고 성능:")
print(f"  • Accuracy:  {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print(f"  • Precision: {val_precision:.4f} ({val_precision*100:.2f}%)")
print(f"  • Recall:    {val_recall:.4f} ({val_recall*100:.2f}%)")
print(f"  • F1-Score:  {val_f1:.4f} ({val_f1*100:.2f}%)")

print(f"\n📊 데이터 분할:")
print(f"  • 학습:  {len(X_train):,}개 (90%)")
print(f"  • 검증:  {len(X_val):,}개 (10%)")
print(f"  • 테스트: {len(df_test):,}개")

print("\n💾 Google Drive에서 파일을 다운로드하거나 확인할 수 있습니다!")
print("\n" + "█"*70 + "\n")

03. 라이브러리 설치
✓ 모든 라이브러리 설치 완료

04. 설정
✓ 모델: klue/bert-base
✓ 디바이스: cuda
✓ 배치 크기: 32
✓ 에포크: 3
✓ 학습률: 2e-05
✓ Validation 비율: 10.0%

05. Dataset 클래스 정의
✓ Dataset 클래스 정의 완료

06. 학습 데이터 로드 및 전처리
✓ 원본 데이터: 4,950개
✓ 결측치 제거 후: 4,950개
✓ 중복 제거 후: 4,619개

📊 클래스 분포:
class
기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
일반 대화           773
Name: count, dtype: int64

07. 라벨 인코딩
✓ 클래스 개수: 5

클래스 매핑:
  0: 갈취 대화                (973개)
  1: 기타 괴롭힘 대화            (1,011개)
  2: 일반 대화                (773개)
  3: 직장 내 괴롭힘 대화          (970개)
  4: 협박 대화                (892개)

08. 학습/검증 데이터 분할 (90/10)
✓ 학습 데이터: 4,157개 (90%)
✓ 검증 데이터: 462개 (10%)

09. 토크나이저 로드
✓ 토크나이저 로드 완료

10. 모델 로드


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ 모델 로드 완료

11. 학습/검증 Dataset 생성
✓ 학습 데이터셋: 4,157개
✓ 검증 데이터셋: 462개
✓ DataLoader 생성 완료

12. 옵티마이저 설정
✓ AdamW 옵티마이저 설정 완료

13. 모델 학습 시작 (학습/검증)



Epoch 1/3 - Val  : 100%|██████████| 15/15 [00:13<00:00,  1.11it/s, loss=0.3659, acc=0.8874]



✓ Epoch 1/3
  Train - Loss: 0.5975 | Accuracy: 0.8015
  Val   - Loss: 0.3659 | Accuracy: 0.8874
  ⭐ 최고 검증 정확도 갱신! (0.8874)



...


✓ Epoch 2/3
  Train - Loss: 0.1975 | Accuracy: 0.9379
  Val   - Loss: 0.2895 | Accuracy: 0.9069
  ⭐ 최고 검증 정확도 갱신! (0.9069)



Epoch 3/3 - Val  : 100%|██████████| 15/15 [00:13<00:00,  1.11it/s, loss=0.3683, acc=0.8896]



✓ Epoch 3/3
  Train - Loss: 0.0960 | Accuracy: 0.9759
  Val   - Loss: 0.3683 | Accuracy: 0.8896

✓ 최고 성능 모델로 복원 완료!

14. 최종 검증 평가 (상세 지표)

✓ 검증 세트 성능 지표:
  Accuracy:  0.8896 (88.96%)
  Precision: 0.8959 (89.59%)
  Recall:    0.8896 (88.96%)
  F1-Score:  0.8885 (88.85%)

✓ 클래스별 성능:
  갈취 대화               : Accuracy=0.7449 (n=98)
  기타 괴롭힘 대화           : Accuracy=0.8119 (n=101)
  일반 대화               : Accuracy=1.0000 (n=77)
  직장 내 괴롭힘 대화         : Accuracy=0.9794 (n=97)
  협박 대화               : Accuracy=0.9438 (n=89)

✓ 혼동 행렬 (Confusion Matrix):
[[73  4  0  3 18]
 [ 7 82  0  5  7]
 [ 0  0 77  0  0]
 [ 0  1  0 95  1]
 [ 1  3  0  1 84]]

15. 테스트 데이터 로드
✓ 테스트 데이터: 500개

16. 테스트 Dataset 생성
✓ 테스트 DataLoader 생성 완료

17. 테스트 데이터 분류

분류 중...



100%|██████████| 16/16 [00:14<00:00,  1.10it/s]


✓ 분류 완료!

18. 결과 데이터프레임 생성
✓ 결과 데이터프레임 생성 완료
✓ 컬럼: ['idx', 'conversation', 'prediction', 'confidence']

19. 결과 저장 (Google Drive)
✓ 예측 결과 저장: /content/drive/My Drive/predictions.csv
✓ 저장된 행 수: 500개

20. 검증 평가 보고서 생성 및 저장
✓ 검증 보고서 저장: /content/drive/My Drive/validation_report.txt

21. 결과 요약 - 테스트 세트 클래스 분포

✓ 테스트 세트 클래스별 분포:
prediction
기타 괴롭힘 대화      138
협박 대화          133
직장 내 괴롭힘 대화    128
갈취 대화           96
일반 대화            5
Name: count, dtype: int64

22. 결과 요약 - 신뢰도 통계

✓ 신뢰도 통계:
  평균: 0.9074
  최고: 0.9967
  최저: 0.2775
  표준편차: 0.1464

23. 결과 요약 - 신뢰도 등급

✓ 신뢰도 등급:
  높음 (≥0.9):  371개 ( 74.2%)
  중간 (0.7~0.9):  66개 ( 13.2%)
  낮음 (<0.7):   63개 ( 12.6%)

24. 상위 10개 예측 결과


  idx  prediction  confidence
t_000       갈취 대화    0.992201
t_001 직장 내 괴롭힘 대화    0.990867
t_002 직장 내 괴롭힘 대화    0.823511
t_003   기타 괴롭힘 대화    0.845130
t_004   기타 괴롭힘 대화    0.745401
t_005       협박 대화    0.992505
t_006       협박 대화    0.993317
t_007   기타 괴롭힘 대화    0.775564
t_008   기타 괴롭힘 대화    0.916282
t_009       갈취 대화   